# Retrieval-Augmented Generation (RAG)

In [2]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [21]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [4]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

/home/vscode/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 31.8MiB/s]


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [13]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [20]:
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")
results = nutrition_qna.query(query_texts=["pregnancy"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(f"Result {i+1}:")
    print(f"Answer: {results['documents'][0]}")
    print("\n")

Result 1:
Answer: ['Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?\n        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.\n\n        This Q&A pair provides information about nutrition and health topics.', 'Question: What health issues can make a pregnancy more challenging?\n        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.\n\n        This Q&A pair provides information about nutrition and health topics.']


Result 2:
Answer: ['Question: What are some possible physical changes that a pregnant woman could experience in the middle months of 

In [31]:
@function_tool
def qa_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Search a verified RAG database for nutrition and health facts. Use this tool 
    for queries regarding food composition, dietary guidelines, and nutrient effects.

    Args:
        query: The nutriotion or health question to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition or health information.
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No verified information found for: {query}"
    
    # Format results for the agent
    formatted_results = []
    formatted_results = []
    # Use a clear separator for the LLM to distinguish between different data sources
    separator = "-" * 40 
    
    # Loop through the retrieved documents
    for i, doc in enumerate(results["documents"][0]):
        # 'doc' is the specific text chunk for this iteration
        entry = (
            f"Result {i+1}:\n"
            f"{doc.strip()}\n"
            f"{separator}"
        )
        formatted_results.append(entry)

    # 4. Join the header and the results
    return f"Information for '{query}':\n\n" + "\n".join(formatted_results)

In [23]:
qa_lookup_tool(query="pregnancy")

'Nutrition Information:\n=== Query: pregnancy === \n Result 1:\n Answer: [\'Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?\\n        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.\\n\\n        This Q&A pair provides information about nutrition and health topics.\', \'Question: What health issues can make a pregnancy more challenging?\\n        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.\\n\\n        This Q&A pair provides information about nutrition and health topics.\', "Question: What are some hormonal changes that take place in a mother\

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [12]:
print(calorie_lookup_tool('bananas'))

Nutrition Information:\nBanana (Fruits): 89.0 calories per 100g
Banana Juice ((Fruit)Juices): 50.0 calories per 100g
Banana Nut Bread (Pastries,Breads&Rolls): 326.0 calories per 100g


In [38]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If you need to find nutrion or health information, use the qa_lookup_tool. 
    When providing nutrition data from the qa_lookup_tool, present the information in the structured format provided by the tool.
    Do not summarize unless the user specifically asks for a summary
    """,
    tools =[calorie_lookup_tool, qa_lookup_tool]
)

In [17]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

- Per 100 g: Banana ≈ 89 kcal; Apple ≈ 52 kcal.
- Estimated total for a typical medium banana (≈118 g) plus a medium apple (≈182 g): ≈ 105 kcal + ≈ 95 kcal = ~200 kcal.
- Note: actual totals depend on the exact sizes/weights.


In [39]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are some potential symptoms of malnutrition in the body?",
    )
    print(result.final_output)

Here are common symptoms that may indicate malnutrition:

- Edema (swelling)
- Hair changes or dyspigmentation
- Angular stomatitis (cracks at the mouth corners)
- Corneal lesions or eye/vision changes
- Swelling or enlargement of glands
- Changes in weight or height; underweight or stunted growth (in children)
- Fatigue, weakness, or reduced activity
- Thinning or dry skin; poor wound healing

Note: Signs can vary and may reflect different nutrient deficiencies; seek professional evaluation if malnutrition is suspected.
